In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [65]:
df = pd.read_csv("dados\\train.csv")

In [66]:
df

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,58640,34,120000,MORTGAGE,5.0,EDUCATION,D,25000,15.95,0.21,Y,10,0
58641,58641,28,28800,RENT,0.0,MEDICAL,C,10000,12.73,0.35,N,8,1
58642,58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2,1
58643,58643,22,30000,RENT,2.0,EDUCATION,A,5000,8.90,0.17,N,3,0


In [67]:
df.duplicated().sum()

0

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

In [ ]:
colunas_originais = df.columns.tolist()

colunas_traduzidas = ['id', 'idade', 'renda_anual', 'status_moradia_atual',
                      'tempo_emprego', 'proposito_emprestimo', 'classificacao_risco', 
                      'valor_total_emprestimo', 'taxa_juros', 'proporcao_emprestimo_renda',
                      'tem_emprestimo_anterior', 'tempo_relacionamento_bancario ', 'status_emprestimo'
                      ]

In [70]:
dicionario_traducao_colunas = {}
for i, k in zip(colunas_originais, colunas_traduzidas):
    dicionario_traducao_colunas[i] = k

In [71]:
df.rename(columns=dicionario_traducao_colunas, inplace=True)
df

,id,idade,renda_anual,status_moradia_atual,tempo_emprego,proposito_emprestimo,classificacao_risco,valor_total_emprestimo,taxa_juros,proporcao_emprestimo_renda,negativado,tempo_relacionamento_bancario,status_emprestimo
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,58640,34,120000,MORTGAGE,5.0,EDUCATION,D,25000,15.95,0.21,Y,10,0
58641,58641,28,28800,RENT,0.0,MEDICAL,C,10000,12.73,0.35,N,8,1
58642,58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2,1
58643,58643,22,30000,RENT,2.0,EDUCATION,A,5000,8.90,0.17,N,3,0


In [72]:
df.drop(columns='id', axis=1, inplace=True)

In [73]:
df

,idade,renda_anual,status_moradia_atual,tempo_emprego,proposito_emprestimo,classificacao_risco,valor_total_emprestimo,taxa_juros,proporcao_emprestimo_renda,negativado,tempo_relacionamento_bancario,status_emprestimo
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,MORTGAGE,5.0,EDUCATION,D,25000,15.95,0.21,Y,10,0
58641,28,28800,RENT,0.0,MEDICAL,C,10000,12.73,0.35,N,8,1
58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2,1
58643,22,30000,RENT,2.0,EDUCATION,A,5000,8.90,0.17,N,3,0


In [74]:
df.isnull().sum()

idade                             0
renda_anual                       0
status_moradia_atual              0
tempo_emprego                     0
proposito_emprestimo              0
classificacao_risco               0
valor_total_emprestimo            0
taxa_juros                        0
proporcao_emprestimo_renda        0
negativado                        0
tempo_relacionamento_bancario     0
status_emprestimo                 0
dtype: int64

In [75]:
df['negativado'] = df['negativado'].apply(lambda x: 1 if x == 'Y' else 0)

df['negativado'].value_counts(1)


negativado
0    0.851616
1    0.148384
Name: proportion, dtype: float64

In [76]:
dicionario_riscos = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7}
df['classificacao_risco'] = df['classificacao_risco'].apply(lambda x: dicionario_riscos[x])
df

,idade,renda_anual,status_moradia_atual,tempo_emprego,proposito_emprestimo,classificacao_risco,valor_total_emprestimo,taxa_juros,proporcao_emprestimo_renda,negativado,tempo_relacionamento_bancario,status_emprestimo
0,37,35000,RENT,0.0,EDUCATION,2,6000,11.49,0.17,0,14,0
1,22,56000,OWN,6.0,MEDICAL,3,4000,13.35,0.07,0,2,0
2,29,28800,OWN,8.0,PERSONAL,1,6000,8.90,0.21,0,10,0
3,30,70000,RENT,14.0,VENTURE,2,12000,11.11,0.17,0,5,0
4,22,60000,RENT,2.0,MEDICAL,1,6000,6.92,0.10,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,MORTGAGE,5.0,EDUCATION,4,25000,15.95,0.21,1,10,0
58641,28,28800,RENT,0.0,MEDICAL,3,10000,12.73,0.35,0,8,1
58642,23,44000,RENT,7.0,EDUCATION,4,6800,16.00,0.15,0,2,1
58643,22,30000,RENT,2.0,EDUCATION,1,5000,8.90,0.17,0,3,0


In [77]:
df['proposito_emprestimo'].unique()

array(['EDUCATION', 'MEDICAL', 'PERSONAL', 'VENTURE', 'DEBTCONSOLIDATION',
       'HOMEIMPROVEMENT'], dtype=object)

In [78]:
dicionario_traducao_proposito_emprestimo =  {
    'EDUCATION': 'educacao',
    'MEDICAL': 'saude',
    'PERSONAL': 'pessoal',
    'VENTURE': 'empreendimento',
    'DEBTCONSOLIDATION': 'consolidacao_divida',
    'HOMEIMPROVEMENT': 'melhoria_residencial'
}

In [79]:
df['proposito_emprestimo'] = df['proposito_emprestimo'].apply(lambda x: dicionario_traducao_proposito_emprestimo[x])
df

,idade,renda_anual,status_moradia_atual,tempo_emprego,proposito_emprestimo,classificacao_risco,valor_total_emprestimo,taxa_juros,proporcao_emprestimo_renda,negativado,tempo_relacionamento_bancario,status_emprestimo
0,37,35000,RENT,0.0,educacao,2,6000,11.49,0.17,0,14,0
1,22,56000,OWN,6.0,saude,3,4000,13.35,0.07,0,2,0
2,29,28800,OWN,8.0,pessoal,1,6000,8.90,0.21,0,10,0
3,30,70000,RENT,14.0,empreendimento,2,12000,11.11,0.17,0,5,0
4,22,60000,RENT,2.0,saude,1,6000,6.92,0.10,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,MORTGAGE,5.0,educacao,4,25000,15.95,0.21,1,10,0
58641,28,28800,RENT,0.0,saude,3,10000,12.73,0.35,0,8,1
58642,23,44000,RENT,7.0,educacao,4,6800,16.00,0.15,0,2,1
58643,22,30000,RENT,2.0,educacao,1,5000,8.90,0.17,0,3,0


In [80]:
df['status_moradia_atual'].unique()

dicionario_traducao_moradia = {
    'RENT': 'aluguel',
    'OWN': 'propria',
    'MORTGAGE': 'hipoteca',
    'OTHER': 'outro'
}


In [81]:
df['status_moradia_atual'] = df['status_moradia_atual'].apply(lambda x: dicionario_traducao_moradia[x])
df


,idade,renda_anual,status_moradia_atual,tempo_emprego,proposito_emprestimo,classificacao_risco,valor_total_emprestimo,taxa_juros,proporcao_emprestimo_renda,negativado,tempo_relacionamento_bancario,status_emprestimo
0,37,35000,aluguel,0.0,educacao,2,6000,11.49,0.17,0,14,0
1,22,56000,propria,6.0,saude,3,4000,13.35,0.07,0,2,0
2,29,28800,propria,8.0,pessoal,1,6000,8.90,0.21,0,10,0
3,30,70000,aluguel,14.0,empreendimento,2,12000,11.11,0.17,0,5,0
4,22,60000,aluguel,2.0,saude,1,6000,6.92,0.10,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,hipoteca,5.0,educacao,4,25000,15.95,0.21,1,10,0
58641,28,28800,aluguel,0.0,saude,3,10000,12.73,0.35,0,8,1
58642,23,44000,aluguel,7.0,educacao,4,6800,16.00,0.15,0,2,1
58643,22,30000,aluguel,2.0,educacao,1,5000,8.90,0.17,0,3,0


In [82]:
df['status_emprestimo'].value_counts(1)

status_emprestimo
0    0.857618
1    0.142382
Name: proportion, dtype: float64

In [88]:
df.groupby('classificacao_risco')['status_emprestimo'].value_counts(1)

classificacao_risco  status_emprestimo
1                    0                    0.950820
                     1                    0.049180
2                    0                    0.897696
                     1                    0.102304
3                    0                    0.864625
                     1                    0.135375
4                    1                    0.593564
                     0                    0.406436
5                    1                    0.625372
                     0                    0.374628
6                    1                    0.610738
                     0                    0.389262
7                    1                    0.818182
                     0                    0.181818
Name: proportion, dtype: float64